# Convolutional Neural Network - MNIST

In [4]:

from neon.backends import gen_backend
from neon.layers import Conv, Affine, Pooling
from neon.initializers import Uniform
from neon.data import ArrayIterator
from neon.transforms.activation import Rectlin, Softmax
from neon.callbacks.callbacks import ProgressBarCallback
from neon.data import MNIST
from neon.layers import GeneralizedCost
from neon.transforms import CrossEntropyMulti
from neon.models import Model
from neon.optimizers import GradientDescentMomentum, RMSProp
from neon.callbacks.callbacks import Callbacks
from neon.transforms import Misclassification, Accuracy

In [5]:
# Step 1: Setup Backend to Use CPU
be = gen_backend(backend='cpu', batch_size=128)

# Step 2: Setup a MNIST DataLoader
MNIST = MNIST()
train_set = MNIST.train_iter
test_set = MNIST.valid_iter

# Step 3: Create an Initializer for the weights
init_uni = Uniform(low=-0.1, high=0.1)

# Step 4: Create the network Architecture 
layers = [Conv(fshape=(5,5,14), init=init_uni, activation=Rectlin()),
          Pooling(fshape=2, strides=2),
          Conv(fshape=(5,5,28), init=init_uni, activation=Rectlin()),
          Pooling(fshape=2, strides=2),
          Affine(nout=500, init=init_uni, activation=Rectlin()),
          Affine(nout=10, init=init_uni, activation=Softmax())]

# Step 5: Create Model Object using the layers defined above
model = Model(layers)

# Step 6: Create our Cost/Loss Function
cost = GeneralizedCost(costfunc=CrossEntropyMulti())

# Step 7: Define our Optimizer
optimizer = GradientDescentMomentum(learning_rate=0.005,
                                    momentum_coef=0.9)

# Step 8: Define our Callback to check progress during Training
callbacks = Callbacks(model, train_set)

# Step 9: Run the model
model.fit(dataset=train_set, cost=cost, optimizer=optimizer,  num_epochs=3, callbacks=callbacks)

# Step 10: Calculate our Error % 
error_pct = 100 * model.eval(test_set, metric=Misclassification())
print('Misclassification error = %.1f%%' % error_pct)
acc_pct = 100 * model.eval(test_set, metric=Accuracy())
print('Test Accuracy = %.1f%%' % acc_pct)


Epoch 0   [Train |████████████████████|  469/469  batches, 0.11 cost, 98.83s]
Epoch 1   [Train |████████████████████|  469/469  batches, 0.08 cost, 99.63s]
Epoch 2   [Train |████████████████████|  469/469  batches, 0.07 cost, 99.00s]
Misclassification error = 2.1%
Train Accuracy = 97.9%
